_New Years Resolutions Analysis_

- _Dataset: contains about 5000 tweets of New Years Resoltuions_
- _Source: Kaggle_

In [4]:
# Importing the libraries required
import pandas as pd

In [12]:
# Reading the dataset and understanding its dimensions
ny_resolutions = pd.read_csv('new_year_resolutions.csv', sep = ';')
ny_resolutions.shape

(5002, 14)

In [20]:
# Setting  custom index for the dataframe
ny_resolutions.set_index('tweet_id', inplace = True)

KeyError: "None of ['tweet_id'] are in the columns"

In [22]:
ny_resolutions['text']

tweet_id
5,50363E+17    #NewYearsResolution :: Read more books, No scr...
5,50363E+17    #NewYearsResolution Finally master @ZJ10 's pa...
5,50362E+17    #NewYearsResolution to stop being so damn perf...
5,50362E+17    My #NewYearsResolution is to help my disabled ...
5,50362E+17    #NewYearsResolution #2015Goals #2015bucketlist...
5,50362E+17      #NewYearsResolution 1. Eat less.\n2.quit lying.
5,50361E+17    My #NewYearsResolution \n\n-Learn how to drive...
5,5036E+17     treat people the way they treat me! _????????#...
5,5036E+17     #NewYearsResolution Stop worrying about irrele...
5,5036E+17     #NewYearsResolution to start a hobby, do it on...
5,50359E+17    #NewYearsResolution Watch Die Hard twice a day...
5,50359E+17         Time to give up tobacco! #NewYearsResolution
5,50359E+17    Theme 2015 is about trusting justice, peace an...
5,50359E+17    RT @kymeshawhite: #NewYearsResolution To grow ...
5,50359E+17    I was going to resolve to procrastinate less, ...
5,50359E+17    R